In [33]:
import sys
sys.path.append("././quaos")
import numpy as np
from basic_functions import int_to_bases, bases_to_int
from paulis import (PauliSum, PauliString, Pauli,)
import sympy as sym
from circuits import Gate, Circuit, Hadamard as H, SUM as CX, PHASE as S

In [84]:
# all two qubit 4 Pauli hamiltonians
r11, r12, r21, r22, r31, r32, r41, r42 = sym.symbols('r11 r12 r21 r22 r31 r32 r41 r42')
s11, s12, s21, s22, s31, s32, s41, s42 = sym.symbols('s11 s12 s21 s22 s31 s32 s41 s42')

M = sym.Matrix([[r11, r12,s11, s12],[r21, r22, s21, s22], [r31, r32, s31, s32], [r41, r42, s41, s42]])
J = sym.Matrix([[0, 0, 1, 0],[0, 0, 0, 1], [1, 0, 0, 0], [0, 1, 0, 0]])
H = sym.Matrix([[0, 0, 1, 0],[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1]])

dims = [2,2]
n_paulis = 4
pauli_string_max_options = np.prod([2,2,2,2])
#print(pauli_string_max_options)
coefficients = [1 for i in range(n_paulis)]
for i in range(pauli_string_max_options):
    for j in range(pauli_string_max_options):
        if i!= j:
            for k in range(pauli_string_max_options):
                if k != i and k != j:
                    for l in range(pauli_string_max_options):
                        if l != i and l != j and l != k:
                            pauli_strings = ['' for i in range(n_paulis)]
                            exponents0 = int_to_bases(i, [2,2,2,2])
                            for m in range(2):
                                r, s = int(exponents0[2*m]), int(exponents0[2*m+1])
                                pauli_strings[0] += f"x{r}z{s} "
                            exponents1 = int_to_bases(j, [2,2,2,2])
                            for m in range(2):
                                r, s = int(exponents1[2*m]), int(exponents1[2*m+1])
                                pauli_strings[1] += f"x{r}z{s} "
                            exponents2 = int_to_bases(k, [2,2,2,2])
                            for m in range(2):
                                r, s = int(exponents2[2*m]), int(exponents2[2*m+1])
                                pauli_strings[2] += f"x{r}z{s} "
                            exponents3 = int_to_bases(l, [2,2,2,2])
                            for m in range(2):
                                r, s = int(exponents3[2*m]), int(exponents3[2*m+1])
                                pauli_strings[3] += f"x{r}z{s}"
                            #print(pauli_strings)
                            P = PauliSum(pauli_strings, weights=coefficients ,dimensions=dims, phases=None, standardise=False)
                            P_sym = sym.Matrix(P.symplectic())

                            # P_c
                            P_c = (M*P_sym.T).T
                            H_P_c = P_c*H.T
                            #display(P_c)
                            #display(P_c*H.T)



                            # Boundary conditions
                            bounds = M.T * J * M - J
                            #display(bounds)

                            # Plug in variables to find minimum of cost function
                            for n in range(2**16):
                                vars = int_to_bases(n, [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2])
                                subs_bounds = bounds.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
                                #display(subs_bounds)
                                if subs_bounds.equals(sym.Matrix.zeros(subs_bounds.rows, subs_bounds.cols)):
                                    subs_H_P_c = H_P_c.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
                                    subs_P_c = P_c.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
                                    dictionary = {matrix_to_string(subs_P_c[0,:]):-coefficients[0], matrix_to_string(subs_P_c[1,:]):-coefficients[1], matrix_to_string(subs_P_c[2,:]):-coefficients[2], matrix_to_string(subs_P_c[3,:]):-coefficients[3]}
                                    #display(dictionary)
                                    dictionary[matrix_to_string(subs_H_P_c[0,:])] += coefficients[0]
                                    dictionary[matrix_to_string(subs_H_P_c[1,:])] += coefficients[1]
                                    dictionary[matrix_to_string(subs_H_P_c[2,:])] += coefficients[2]
                                    dictionary[matrix_to_string(subs_H_P_c[3,:])] += coefficients[3]
                                    #display(dictionary)
                                    cost = sum(value for value in dictionary.values())
                                    if cost == 0:
                                        display(P_sym)
                                        display(subs_P_c)
                                        subs_M = M.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
                                        display(subs_M)

                            

Matrix([
[0,   0, 0,   0],
[0,   0, 0, 1.0],
[0, 1.0, 0,   0],
[0, 1.0, 0, 1.0]])

Matrix([
[  0, 0,   0, 0],
[1.0, 0,   0, 0],
[  0, 0, 1.0, 0],
[1.0, 0, 1.0, 0]])

Matrix([
[0, 0, 0, 1],
[0, 0, 1, 0],
[0, 1, 0, 0],
[1, 0, 0, 0]])

Matrix([
[0,   0, 0,   0],
[0,   0, 0, 1.0],
[0, 1.0, 0,   0],
[0, 1.0, 0, 1.0]])

Matrix([
[0,   0, 0,   0],
[0, 1.0, 0,   0],
[0,   0, 0, 1.0],
[0, 1.0, 0, 1.0]])

...

Matrix([
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1],
[1, 0, 0, 0]])

Matrix([
[0,   0, 0,   0],
[0,   0, 0, 1.0],
[0, 1.0, 0,   0],
[0, 1.0, 0, 1.0]])

Matrix([
[  0, 0,   0, 0],
[  0, 0, 1.0, 0],
[1.0, 0,   0, 0],
[1.0, 0, 1.0, 0]])

Matrix([
[0, 1, 0, 0],
[1, 0, 0, 0],
[0, 0, 0, 1],
[0, 0, 1, 0]])

KeyboardInterrupt: 

In [34]:
# all two qubit 4 Pauli hamiltonians
r11, r12, r21, r22, r31, r32, r41, r42 = sym.symbols('r11 r12 r21 r22 r31 r32 r41 r42')
s11, s12, s21, s22, s31, s32, s41, s42 = sym.symbols('s11 s12 s21 s22 s31 s32 s41 s42')

M = sym.Matrix([[r11, r12,s11, s12],[r21, r22, s21, s22], [r31, r32, s31, s32], [r41, r42, s41, s42]])
J = sym.Matrix([[0, 0, 1, 0],[0, 0, 0, 1], [1, 0, 0, 0], [0, 1, 0, 0]])
#H = sym.Matrix([[0, 0, 1, 0],[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1]])

dims = [2,2]
n_paulis = 4
pauli_string_max_options = np.prod([2,2,2,2])
#print(pauli_string_max_options)
coefficients = [1 for i in range(n_paulis)]

i = 0
j = 1
k = 2
l = 3
pauli_strings = ['' for i in range(n_paulis)]
exponents0 = int_to_bases(i, [2,2,2,2])
for m in range(2):
    r, s = int(exponents0[2*m]), int(exponents0[2*m+1])
    pauli_strings[0] += f"x{r}z{s} "
exponents1 = int_to_bases(j, [2,2,2,2])
for m in range(2):
    r, s = int(exponents1[2*m]), int(exponents1[2*m+1])
    pauli_strings[1] += f"x{r}z{s} "
exponents2 = int_to_bases(k, [2,2,2,2])
for m in range(2):
    r, s = int(exponents2[2*m]), int(exponents2[2*m+1])
    pauli_strings[2] += f"x{r}z{s} "
exponents3 = int_to_bases(l, [2,2,2,2])
for m in range(2):
    r, s = int(exponents3[2*m]), int(exponents3[2*m+1])
    pauli_strings[3] += f"x{r}z{s}"
#print(pauli_strings)
P = PauliSum(pauli_strings, weights=coefficients ,dimensions=dims, phases=None, standardise=False)
P_sym = sym.Matrix(np.array(P.symplectic(),dtype=np.int32))
display(P_sym)
#display(P_sym*H.T)

# P_c
P_c = (M*P_sym.T).T
#H_P_c = P_c*H.T
#display(P_c)
#display(P_c*H.T)



# Boundary conditions
bounds = M.T * J * M - J
display(bounds)
g = H(1,2)
C = Circuit(dims,gates=[g])
# Plug in variables to find minimum of cost function
#for n in range(2**16):
n = 1632
vars = int_to_bases(n, [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2])
subs_bounds = bounds.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
#display(subs_bounds)
if subs_bounds.equals(sym.Matrix.zeros(subs_bounds.rows, subs_bounds.cols)):
    #print(n)
    #subs_H_P_c = H_P_c.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
    print(subs_P_c[0,0:2].tolist()[0],subs_P_c[0,2:4].tolist())
    print(type(np.array(subs_P_c[0,0:2].tolist()[0])))
    print(PauliString(np.array(subs_P_c[0,0:2].tolist()[0]),np.array(subs_P_c[0,2:4].tolist()[0]),dimensions=dims))
    subs_P_c = P_c.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
    subs_P_c_PS = PauliSum([PauliString(np.array(subs_P_c[i,0:2].tolist()[0]),np.array(subs_P_c[i,2:4].tolist()[0]),dimensions=dims) for i in range(4)],dimensions=dims, phases=None, standardise=False)
    H_P_c_PS = C.act(subs_P_c_PS)
    subs_H_P_c = sym.Matrix(np.array(H_P_c_PS.symplectic(),dtype=np.int32))

    dictionary = {matrix_to_string(subs_P_c[0,:]):-coefficients[0], matrix_to_string(subs_P_c[1,:]):-coefficients[1], matrix_to_string(subs_P_c[2,:]):-coefficients[2], matrix_to_string(subs_P_c[3,:]):-coefficients[3]}
    #display(dictionary)
    dictionary[matrix_to_string(subs_H_P_c[0,:])] += H_P_c_PS.phases[0]*coefficients[0]
    dictionary[matrix_to_string(subs_H_P_c[1,:])] += H_P_c_PS.phases[1]*coefficients[1]
    dictionary[matrix_to_string(subs_H_P_c[2,:])] += H_P_c_PS.phases[2]*coefficients[2]
    dictionary[matrix_to_string(subs_H_P_c[3,:])] += H_P_c_PS.phases[3]*coefficients[3]
    #display(dictionary)
    cost = sum(value for value in dictionary.values())
    if cost == 0:
        display(subs_P_c)
        subs_M = M.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
        display(subs_M)
#print(cost)
#display(subs_P_c)
#display(subs_H_P_c[1,:])
#display(subs_H_P_c)
#display(subs_bounds)


Matrix([
[0, 0, 0, 0],
[0, 0, 0, 1],
[0, 1, 0, 0],
[0, 1, 0, 1]])

Matrix([
[                    2*r11*r31 + 2*r21*r41,     r11*r32 + r12*r31 + r21*r42 + r22*r41, r11*s31 + r21*s41 + r31*s11 + r41*s21 - 1,     r11*s32 + r21*s42 + r31*s12 + r41*s22],
[    r11*r32 + r12*r31 + r21*r42 + r22*r41,                     2*r12*r32 + 2*r22*r42,     r12*s31 + r22*s41 + r32*s11 + r42*s21, r12*s32 + r22*s42 + r32*s12 + r42*s22 - 1],
[r11*s31 + r21*s41 + r31*s11 + r41*s21 - 1,     r12*s31 + r22*s41 + r32*s11 + r42*s21,                     2*s11*s31 + 2*s21*s41,     s11*s32 + s12*s31 + s21*s42 + s22*s41],
[    r11*s32 + r21*s42 + r31*s12 + r41*s22, r12*s32 + r22*s42 + r32*s12 + r42*s22 - 1,     s11*s32 + s12*s31 + s21*s42 + s22*s41,                     2*s12*s32 + 2*s22*s42]])

[0, 0] [[0, 0]]
<class 'numpy.ndarray'>
x0z0 x0z0 


TypeError: Unsupported type <class 'paulis.pauli_sum.PauliSum'> for Gate.act. Expected Pauli, PauliString or PauliSum.

In [35]:
print(type(np.array(subs_P_c[0,0:2].tolist()[0])))
print(type(np.array([0,1])))
P_test = PauliString(np.array(subs_P_c[0,0:2].tolist()[0]),np.array(subs_P_c[0,2:4].tolist()[0]),dimensions=dims)
PS_test = PauliSum([P_test,P_test],dimensions=dims, phases=None, standardise=False)
C.act(PS_test)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


TypeError: Unsupported type <class 'paulis.pauli_sum.PauliSum'> for Gate.act. Expected Pauli, PauliString or PauliSum.

In [75]:
def matrix_to_string(matrix):
    """Convert a SymPy matrix with 0s and 1s into a string."""
    return ''.join(map(str, matrix.tolist()[0]))


In [18]:
xs = [0,1,2,3]
''.join(str(x) for x in xs)

'0123'

In [23]:
x, y = sym.symbols('x y')
A = {'0': x, '1': y}
A['0'] += y
A

{'0': x + y, '1': y}

In [48]:
sym.Matrix(np.array([[0, 0, 0, 0],[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],dtype=np.int32)).is_zero

False

In [52]:
A = sym.Matrix([[0, 0], [0, 0]])

# Check if all elements are zero
A.equals(sym.Matrix.zeros(A.rows, A.cols))


True